In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [2]:
df2 = pd.read_csv('20220720_F_TrackingDebt_BI.csv')
# dk = df2.tiennocongty > 1000
# df2 = df2[dk].copy()
df2.DateOfOrder = convert_to_datetime(df2.DateOfOrder)
df2.DueDate = convert_to_datetime(df2.DueDate)
df2.deli_last_updated = pd.to_datetime(df2.deli_last_updated, dayfirst=True)
# dk1=df2['SubChannel'].isin("")
# dk2=True

# *------------------*
#24032022
df2.columns = lower_col(df2)
dk1 = df2.subchannel.isin(["NT","PK","SI"])
dk2 = df2.paymentsform == "TM"
# dka = df2.terms.isin( ["Gối Đầu 30 Pha Nam","Thu tiền ngay không có VP PN","Thu tiền ngay có VP PN","Gối 1 Đơn Hàng (trong 30 ngày)"] )
dkfinal1 = dk1&dk2
dk3 = df2.subchannel == 'CHUOI'
dk4 = df2.termstype == "Thanh Toán Ngay"
dk5 = df2.paymentsform == "TM"
dkfinal2 = dk3&dk4&dk5
dk6 = df2.subchannel.isin(["INS1","INS2","INS3"])
df2['debtincharge_v2']=np.where(dkfinal1 | dkfinal2, "MDS", \
    np.where(dk6, "INS", "CS"))
# *------------------*

dk1 = df2.ordernbr == 'DH122018-17643'
dk2 = df2.branchid == 'MR0001'
df2.debtincharge_v2 = np.where(dk1&dk2, "CS", df2.debtincharge_v2)
# *------------------*

#update Hanh input
dk1 = df2.ordernbr == 'DH062018-13754'
dk2 = df2.branchid == 'MR0003'
df2.debtincharge_v2 = np.where(dk1&dk2, "CS", df2.debtincharge_v2)
#
dk=df2['debtincharge_v2'] == 'MDS'
df2 = df2[dk].copy()
df2['inserted_at'] = datetime.now()
# BQ first 
bq_values_insert(df2, "f_tracking_debt_hanhngu", 3)

1it [00:36, 36.40s/it]
